In [14]:
import zipfile
import pandas as pd

# California

In [15]:
df_ca = pd.read_pickle("../data/ca_cleaned.pkl", compression= "gzip")

In [16]:
df_ca.head()

,date,date_dt,county_name,subject_race,subject_sex,search_conducted,search_basis,outcome
0,2009-07-01,2009-07-01,Stanislaus County,other,male,False,no_search,unknown
1,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,False,no_search,summons
2,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,True,other,summons
3,2009-07-01,2009-07-01,Stanislaus County,white,female,False,no_search,summons
4,2009-07-01,2009-07-01,Stanislaus County,hispanic,male,True,other,summons


In [17]:
df_ca["date"] = pd.to_datetime(df_ca["date"])

In [18]:
df_ca["year"] = df_ca["date"].dt.year

There are some spelling mistakes in county names and suffixe County to delete at the end of the string.

In [19]:
df_ca['county_name'] = df_ca['county_name'].str[:-7]
df_ca['county_name'] = df_ca['county_name'].apply(lambda x : x.replace("Siskyou", "Siskiyou"))
df_ca['county_name'] = df_ca['county_name'].apply(lambda x : x.replace("Santa Barbera", "Santa Barbara"))

## Ethnicity

In [20]:
count_year_ethnicity_ca = df_ca.groupby(["year", "county_name", "subject_race"]).count()["date"]
count_year_ethnicity_ca = pd.DataFrame(count_year_ethnicity_ca)
count_year_ethnicity_ca = count_year_ethnicity_ca.rename(columns = {"date": "nb_arrest"})

count_year_ethnicity_ca.head(10)


count
year county_name subject_race                 
2009 Alameda     asian/pacific islander  13886
                 black                   20976
                 hispanic                25689
                 other                   13071
                 white                   40816
     Alpine      asian/pacific islander    341
                 black                     125
                 hispanic                  661
                 other                     348
                 white                    5630

In [29]:
count_year_ethnicity_ca = count_year_ethnicity_ca.reset_index()

## Gender

In [21]:
count_year_gender_ca = df_ca.groupby(["year", "county_name", "subject_sex"]).count()["date"]
count_year_gender_ca = pd.DataFrame(count_year_gender_ca)
count_year_gender_ca = count_year_gender_ca.rename(columns = {"date": "nb_arrest"})

count_year_gender_ca.head(10)

count
year county_name subject_sex       
2009 Alameda     female       34786
                 male         79651
     Alpine      female        1895
                 male          5210
     Amador      female        2239
                 male          4773
     Butte       female        6795
                 male         11053
     Calaveras   female        3282
                 male          6266

In [28]:
count_year_gender_ca = count_year_gender_ca.reset_index()

In [61]:
count_year_ethnicity_ca.to_pickle("../data/count_year_ethnicity_ca.pkl", compression="gzip")
count_year_gender_ca.to_pickle("../data/count_year_gender_ca.pkl", compression="gzip")


# Population

In [43]:
population_ca = pd.read_csv('../data/population_ca.csv')
population_ca.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [44]:
population_ca = population_ca.groupby('County')['pop_april_2010'].agg(['sum'])
population_ca.rename(columns={'sum' : 'population'},inplace = True)
relative_count_ethnicity_ca = pd.merge(count_year_ethnicity_ca, population_ca, left_on = 'county_name', right_on = 'County')
relative_count_ethnicity_ca['relative_arrest'] = relative_count_ethnicity_ca['count'] / relative_count_ethnicity_ca['population']

In [65]:
relative_count_gender_ca = pd.merge(count_year_gender_ca, population_ca, left_on = 'county_name', right_on = 'County')
relative_count_gender_ca['relative_arrest'] = relative_count_gender_ca['count'] / relative_count_gender_ca['population']

#### Ethni

In [59]:
ethni_white_ca = relative_count_ethnicity_ca.query("subject_race == 'white'")
ethni_black_ca = relative_count_ethnicity_ca.query("subject_race == 'black'")
ethni_asian_ca = relative_count_ethnicity_ca.query("subject_race == 'asian/pacific islander'")
ethni_hispanic_ca = relative_count_ethnicity_ca.query("subject_race == 'hispanic'")

In [60]:
ethni_white_ca.to_csv('../data/arrest_white_ca.csv')
ethni_black_ca.to_csv('../data/arrest_black_ca.csv')
ethni_asian_ca.to_csv('../data/arrest_asian_ca.csv')
ethni_hispanic_ca.to_csv('../data/arrest_hispanic_ca.csv')

#### Gender

In [67]:
female_ca = relative_count_gender_ca.query("subject_sex == 'female'")
male_ca = relative_count_gender_ca.query("subject_sex == 'male'")

In [68]:
female_ca.to_csv('../data/arrest_female_ca.csv')
male_ca.to_csv('../data/arrest_male_ca.csv')

-----

# Texas